In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
!pip -q install autogluon

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-pubsub 2.17.1 requires grpcio<2.0dev,>=1.51.3, but you have grpcio 1.50.0 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
path='/kaggle/input/wind-speed/open/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
train=train.drop('ID',axis=1)
test=test.drop('ID',axis=1)

In [5]:
column_mapping = {
    "월": "Month",
    "일": "Day",
    "측정 시간대": "Measurement Time",
    "섭씨 온도(°⁣C)": "Temperature",
    "절대 온도(K)": "Absolute Temperature",
    "이슬점 온도(°C)": "Dew Point Temperature",
    "상대 습도 (%)": "Relative Humidity",
    "대기압(mbar)": "Atmospheric Pressure",
    "포화 증기압(mbar)": "Saturated Vapor Pressure",
    "실제 증기압(mbar)": "Actual Vapor Pressure",
    "증기압 부족량(mbar)": "Vapor Pressure Deficit",
    "수증기 함량 (g/kg)": "Water Vapor Content",
    "공기 밀도 (g/m**3)": "Air Density",
    "풍향 (deg)": "Wind Direction"
}

train.rename(columns=column_mapping, inplace=True)
train.rename(columns={'풍속 (m/s)':'wind speed'}, inplace=True)
test.rename(columns=column_mapping, inplace=True)

In [6]:
train=train.drop('Absolute Temperature',axis=1)
test=test.drop('Absolute Temperature',axis=1)

In [7]:
# train['Dew Point abs Temperature']=train["Dew Point Temperature"]+(train["Absolute Temperature"]-train["Temperature"])
# test['Dew Point abs Temperature']=test["Dew Point Temperature"]+(test["Absolute Temperature"]-test["Temperature"])

In [8]:
train.head()

,Month,Day,Measurement Time,Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Pressure,Saturated Vapor Pressure,Actual Vapor Pressure,Vapor Pressure Deficit,Water Vapor Content,Air Density,Wind Direction,wind speed
0,7,2,저녁,13.97,9.84,76.1,992.08,15.98,12.16,3.82,7.66,1198.06,155.60,1.61
1,8,21,오전,16.94,12.14,73.3,991.07,19.33,14.17,5.16,8.94,1183.67,177.00,1.68
2,11,1,저녁,9.76,5.40,74.2,988.71,12.10,8.98,3.12,5.67,1213.22,146.20,0.73
3,12,28,오전,5.27,2.71,83.5,1014.25,8.89,7.43,1.47,4.57,1265.48,264.50,2.71
4,9,26,오후,17.35,12.68,74.0,995.77,19.84,14.68,5.16,9.22,1187.40,19.34,1.00


In [9]:
test

,Month,Day,Measurement Time,Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Pressure,Saturated Vapor Pressure,Actual Vapor Pressure,Vapor Pressure Deficit,Water Vapor Content,Air Density,Wind Direction
0,3,24,새벽,4.28,-0.21,72.5,984.48,8.30,6.02,2.28,3.81,1233.29,251.80
1,9,24,저녁,13.40,10.36,81.8,996.98,15.40,12.59,2.80,7.89,1206.20,225.60
2,5,28,저녁,19.89,14.95,73.2,984.83,23.26,17.03,6.23,10.82,1163.06,10.39
3,1,17,저녁,-2.88,-4.47,88.7,998.02,4.94,4.38,0.56,2.73,1284.19,260.20
4,10,22,오후,6.97,4.36,83.4,987.00,10.01,8.35,1.66,5.28,1223.47,262.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15673,3,16,새벽,1.61,-0.49,85.9,1001.26,6.86,5.89,0.97,3.67,1266.62,56.18
15674,9,5,오후,18.27,13.35,73.0,990.98,21.03,15.35,5.68,9.69,1177.63,265.30
15675,8,8,새벽,16.08,12.25,78.0,994.97,18.30,14.28,4.03,8.97,1191.84,189.80
15676,2,4,새벽,3.07,-0.95,74.8,994.31,7.62,5.70,1.92,3.57,1251.24,269.70


In [10]:
time_limit=int(3600*11.5)
excluded_model_types = ['KNN', 'NN_TORCH']

*  ['best_quality', 'high_quality', 'good_quality', 'medium_quality', 'optimize_for_deployment', 'ignore_text', 'interpretable', 'experimental_best_quality', 'experimental_extreme_quality', 'experimental_zeroshot_hpo', 'experimental_zeroshot_hpo_hybrid']

In [11]:
predictor = TabularPredictor(label='wind speed', problem_type='regression',
                             eval_metric='mean_absolute_error',
                             sample_weight='auto_weight').fit(train,presets='experimental_zeroshot_hpo_hybrid',fit_weighted_ensemble = True,
                                                              excluded_model_types=excluded_model_types, num_gpus=2, time_limit=time_limit)

No path specified. Models will be saved in: "AutogluonModels/ag-20230724_134259/"
Presets specified: ['experimental_zeroshot_hpo_hybrid']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 41400s
AutoGluon will save models to "AutogluonModels/ag-20230724_134259/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jul 21 03:39:30 UTC 2023
Disk Space Avail:   20.94 GB / 20.96 GB (99.9%)
Train Data Rows:    36581
Train Data Columns: 13
Label Column: wind speed
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    32455.1 MB
	Train Data (Original)  Memory Usage: 6.66 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of th

In [12]:
preds = predictor.predict(test)

In [13]:
submit = pd.read_csv(path+'sample_submission.csv')

In [14]:
submit['풍속 (m/s)']=preds
submit.to_csv('/kaggle/working/submission1.csv', index=False)